In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
#Vertical resolution used : 
0.02
0.1
0.5
1.25010428797
8.33294983235
32.8082236775

32.8082236775

In [3]:
import sys, os

sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools','meshing_ats'))
import meshing_ats

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal

In [4]:
xl =pd.ExcelFile('Topo_Profiles_Barrow.xlsx')

In [5]:
xl.sheet_names
print (xl.sheet_names)
data = xl.parse(sheet_name='Transect B, Area C')
keys = data.keys()
slice_at_right = False

[u'Transect A, Polygon near ERT', u'Transect B, Polygon near ERT', u'Transect Area B', u'Transect A, Area C', u'Transect B, Area C', u'Repeat ERT Transect']


In [6]:
plt.plot(data['Distance_m'], data['Z'],'.')
plt.ylabel('Elevation')
plt.title('Topography, Polygon Area C')
plt.xlabel('Distance [m] (trough-to-trough)')
plt.show()

In [ ]:
def OLD_MESHES_FEB13():
    # smooth the topographic data : September 24
    length = len(data['Distance_m'])
    x = []
    z = []

    if (slice_at_right):
        x = np.array(data['Distance_m'][length/2:])
        z = np.array(data['Z'][length/2:])
    else:
        x = np.array(data['Distance_m'][:46])
        z = np.array(data['Z'][:46])

    z = [z1 - 0.1127955 for z1 in z] # match the trough's elevation with the "trough elevation" from water-table data 


    #z_smooth = scipy.signal.savgol_filter(Z1, window_length=17, polyorder=3, mode='constant',cval=4.7)
    lower_it = 0.05
    lower_center = 0.03
    if lower_it > 0 and lower_center <= 0: #revert sign to execute
        x1 = [x[0], x[10]]
        z1 = [z[0], z[10]-lower_it]
        x2 = [x[10], x[20], x[-1]]
        z2 = [z[10]-lower_it, z[20]-lower_it, z[-1]]
        cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
        cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
    elif lower_center > 0: #revert sign to execute
        x1 = [x[0], x[10]]
        z1 = [z[0], z[10]-lower_it]
        x2 = [x[10], x[20], x[-1]]
        z2 = [z[10]-lower_it, z[20]-lower_it, z[-1]-lower_center]
        cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
        cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
    else:
        x1 = [x[0], x[10]]
        z1 = [z[0], z[10]]
        x2 = [x[10], x[22], x[-1]]
        z2 = [z[10], z[22], z[-1]]
        cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
        cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')

    updated6 = False
    if updated6 == True:
        x1 = [x[0], x[10]]
        z1 = [z[0], 4.81]
        x2 = [x[10], x[20], x[-1]]
        z2 = [4.81, z[20]-0.08, 4.78]
        cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
        cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')


    z_smooth = np.concatenate((cs1(x[:11]), cs2(x[11:])))

    if (slice_at_right):
        z = z[::-1]
        z_smooth = z_smooth[::-1]
        x = x[::-1]

    z_smooth = np.array([round(s,4) for s in z_smooth])

    #plt.plot(x, z, '.', color='r')
    plt.plot(x, z_smooth, '-.', color = 'k')
    plt.plot(x1,z1,'*', color='c')
    plt.plot(x2,z2,'*', color='c')
    #plt.plot(x3,z3,'*', color='c')
    #plt.plot(x4,z4,'.', color='g')
    plt.ylabel('Elevation [m]')
    plt.xlabel('Distance [m]')
    plt.title('Mesh, Polygon Area C')
    plt.show()
    #print len(x), lmid

    #print 'Seepage face high at the center ', max(z_smooth - z_smooth[-1])

In [12]:
# smooth the topographic data : February 13 and trough cell inward oriented
length = len(data['Distance_m'])
x = []
z = []

if (slice_at_right):
    x = np.array(data['Distance_m'][length/2:])
    z = np.array(data['Z'][length/2:])
else:
    x = np.array(data['Distance_m'][:46])
    z = np.array(data['Z'][:46])

z = [z1 - 0.1127955 for z1 in z] # match the trough's elevation with the "trough elevation" from water-table data 
A1 = []
A1 = np.insert(x, 0, -x[1])
x = np.array([a + x[1] for a in A1])
z = np.insert(z,0, z[0]+.0000)

#z_smooth = scipy.signal.savgol_filter(Z1, window_length=17, polyorder=3, mode='constant',cval=4.7)
lower_it = 0.05
lower_center = 0.03
if lower_it > 0 and lower_center <= 0: #revert sign to execute
    x1 = [x[0], x[10]]
    z1 = [z[0], z[10]-lower_it]
    x2 = [x[10], x[20], x[-1]]
    z2 = [z[10]-lower_it, z[20]-lower_it, z[-1]]
    cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
elif lower_center > 0: #revert sign to execute 
    x1 = [x[0], x[1], x[10]]
    z1 = [z[0], z[1], z[10]-lower_it]
    x2 = [x[10], x[20], x[-1]]
    z2 = [z[10]-lower_it, z[20]-lower_it, z[-1]-lower_center]
    cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
else:
    x1 = [x[0], x[10]]
    z1 = [z[0] , z[10]]
    x2 = [x[10], x[22], x[-1]]
    z2 = [z[10], z[22], z[-1]]
    cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')

updated6 = False
if updated6 == True:
    x1 = [x[0], x[10]]
    z1 = [z[0], 4.81]
    x2 = [x[10], x[20], x[-1]]
    z2 = [4.81, z[20]-0.08, 4.78]
    cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')


z_smooth = np.concatenate((cs1(x[:11]), cs2(x[11:])))

if (slice_at_right):
    z = z[::-1]
    z_smooth = z_smooth[::-1]
    x = x[::-1]

z_smooth = np.array([round(s,4) for s in z_smooth])

#plt.plot(x, z, '.', color='r')
plt.plot(x, z_smooth, '-*', color = 'k')
plt.plot(x1,z1,'*', color='c')
plt.plot(x2,z2,'*', color='c')
#plt.plot(x3,z3,'*', color='c')
#plt.plot(x4,z4,'.', color='g')
plt.ylabel('Elevation [m]')
plt.xlabel('Distance [m]')
plt.title('Mesh, Polygon Area C')
plt.show()
#print len(x), lmid

#print 'Seepage face high at the center ', max(z_smooth - z_smooth[-1])

In [ ]:
generic_twopiewedges = False
updated_topo2 = False
if (generic_twopiewedges):
    x = np.array(data['Distance_m'])
    z = np.array(data['Z'])
    z = [z1 - 0.1127955 for z1 in z]
    x1 = [x[0], x[10]]
    z1 = [z[0], z[10]]
    lmid = length/2
    
    x2 = [x[10], x[20], x[lmid]]
    z2 = [z[10], z[20]-0.02, z[lmid]-0.01]
    
    print z[0], z[lmid],  z[0] - z[lmid]
    x3 = [x[lmid],x[lmid+ 26]]
    z3 = [z[lmid]-0.01, z[lmid+26] - 0.035]
    x4 = [x[lmid+26], x[-1]]
    z4 = [z[lmid+26] - 0.035, z[0] ]
    if (updated_topo2):
        x1 = [x[0], x[10]]
        z1 = [z[0], z[10]-0.045]
        x2 = [x[10]-0.045, x[lmid]]
        z2 = [z[10] - 0.045, z[lmid]-0.01]
    cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
    cs3 = scipy.interpolate.CubicSpline(x3,z3, bc_type='clamped')
    cs4 = scipy.interpolate.CubicSpline(x4,z4, bc_type='clamped')

z_smooth = np.concatenate((cs1(x[:11]), cs2(x[11:lmid]), cs3(x[lmid:lmid+26]), cs4(x[lmid+26:])))

In [ ]:
print len(z)/2

In [ ]:
# turn on for single pie wedge original mesh (updated-topo, see Feb7 plots)
#z_smooth = np.concatenate((cs1(x[:11]), cs2(x[11:lmid])))
#x = x[:len(z_smooth)]

updated_topo3 = True
shift = 2
if (updated_topo3):
    print 'upudated_topo3: ', updated_topo3
    x = np.array(data['Distance_m'])
    z = np.array(data['Z'])
    length= len(x) 
    lmid = length/2 + 8
    z = [z1 - 0.1127955 for z1 in z]
    
    x1 = [x[0], x[10]]
    z1 = [z[0], z[10]]
    x2 = [x[10]-0.045, x[20], x[lmid]]
    z2 = [z[10], z[20]-0.02, z[lmid -10]-0.01]
    cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
    z_smooth = np.concatenate((cs1(x[:11]), cs2(x[11:lmid])))
    x = x[:lmid]
    plt.plot(x, z_smooth, '-.', color = 'k')
    plt.plot(x1,z1,'*', color='c')
    plt.plot(x2,z2,'*', color='c')
   
    plt.ylabel('Elevation [m]')
    plt.xlabel('Distance [m]')
    plt.title('Mesh, Polygon Area C')
    plt.show()

# this is no xx-updated4.exo mesh
updated_topo4 = False

if (updated_topo4):
    print 'upudated_topo4: ', updated_topo4
    x = np.array(data['Distance_m'])
    z = np.array(data['Z'])
    length= len(x) 
    lmid = length/2 + 7
    z = [z1 - 0.1127955 for z1 in z]
    
    x1 = [x[0], x[10]]
    z1 = [z[0], z[10]]
    x2 = [x[10]-0.045, x[20], x[lmid]]
    z2 = [z[10], z[20]-0.02, 4.78]
    cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
    z_smooth = np.concatenate((cs1(x[:11]), cs2(x[11:lmid])))
    x = x[:lmid]
    plt.plot(x, z_smooth, '-.', color = 'k')
    plt.plot(x1,z1,'*', color='c')
    plt.plot(x2,z2,'*', color='c')
   
    plt.ylabel('Elevation [m]')
    plt.xlabel('Distance [m]')
    plt.title('Mesh, Polygon Area C')
    plt.show()

In [ ]:
#trough
observe = 'rim'
xloc = 0
zv = z_smooth
mid_pt = (zv[0] + zv[1])/2.

depths_t = [0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6,0.7,0.8, 1.0, 1.25, 1.50]
depths_r = [0.02, 0.05, 0.08, 0.1, 0.15, 0.2, 0.25, 0.30, 0.34, 0.35, 0.4, 0.5, 0.6,0.7,0.8, 1.0, 1.25, 1.50]
depths_c = [0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.42, 0.5, 0.6,0.7,0.8, 1.0, 1.25, 1.50]
if observe == 'trough':
    z_location = [zv[1] - v for v in depths_t] #taking z at zv[1] to make sure the observed point lie within above cell
    xloc = np.where(zv == zv[0])
    xloc = x[xloc]
elif observe == 'rim':
    z_location = [max(zv) - v for v in depths_r]
    xloc = np.where(zv == max(zv))
elif observe == 'center':
    z_location = [zv[-2] - v for v in depths_c]
print round(z_smooth[10],5), max(zv)
print np.round(z_location,3)
print  4.8972 -0.02, xloc, z_smooth[1]

In [ ]:
# Two pie wedges connected at the center
# smooth the topographic data
two_piewedge = False
if two_piewedge == True:
    length = len(data['Distance_m'])
    x = []
    z = []


    x = np.array(data['Distance_m'])
    z = np.array(data['Z'])

    z_smooth = scipy.signal.savgol_filter(z, window_length=15, polyorder=3, mode='nearest')

    z_smooth = scipy.signal.savgol_filter(z_smooth, window_length=15, polyorder=3, mode='nearest')
    z_smooth = scipy.signal.savgol_filter(z_smooth, window_length=5, polyorder=3, mode='nearest')

    plt.plot(x, z, '.')
    plt.plot(x, z_smooth, 'k-')
    plt.ylabel('Elevation')
    plt.title('Mesh, Polygon Area C')
    plt.show()
    
    

In [8]:
import math
def Distance(x,y):
    d = np.zeros(len(x))
    for i in range(len(x)):
        d[i] = math.hypot(x[-1] - x[i], y[-1] - y[i])
    return d

In [9]:
print x, z_smooth
print len(x), len(z_smooth)

[ 0.          0.27521822  0.55043644  0.82565466  1.10087288  1.37609111
  1.65130933  1.92652755  2.20174577  2.47696399  2.75218221  3.02740043
  3.30261865  3.57783688  3.8530551   4.12827332  4.40349154  4.67870976
  4.95392798  5.2291462   5.50436442  5.77958265  6.05480087  6.33001909
  6.60523731  6.88045553  7.15567375  7.43089197  7.70611019  7.98132841
  8.25654664  8.53176486  8.80698308  9.0822013   9.35741952  9.63263774
  9.90785596 10.18307418 10.45829241 10.73351063 11.00872885 11.28394707
 11.55916529 11.83438351 12.10960173 12.38481995 12.66003817] [4.7172 4.7172 4.7229 4.7348 4.751  4.7697 4.7891 4.8074 4.8228 4.8333
 4.8372 4.8369 4.836  4.8347 4.8329 4.8307 4.8283 4.8257 4.8229 4.82
 4.8172 4.8144 4.8117 4.8091 4.8066 4.8042 4.8018 4.7996 4.7974 4.7953
 4.7934 4.7915 4.7897 4.7881 4.7865 4.7851 4.7838 4.7825 4.7815 4.7805
 4.7796 4.7789 4.7783 4.7778 4.7775 4.7773 4.7772]
47 47


In [10]:
reload(meshing_ats)
site_area = 'C'
if site_area == 'A':
    arclen = 16 
elif site_area == 'B':
    arclen = 6 
elif site_area == 'C':
    arclen = 12
elif site_area == 'D':
    arclen = 10

m2 = meshing_ats.Mesh2D.from_Transect_MakePieWedge(x,z_smooth, arclen, curved=False)
#m2 = meshing_ats.Mesh2D.from_Transect_MakeTwoPieWedges(x,z_smooth, arclen)
#m2 =meshing_ats.Mesh2D.read_VTK_Unstructured('column.vtk')
m2.plot()
print site_area, arclen

(94, 3) 2
C 12


In [ ]:
print z_smooth
maxloc = int(xloc[0])
trough_mid = (z_smooth[0] + z_smooth[maxloc]) / 2.0 #xloc is the location of the max z
trough = [x1 for x1 in z_smooth[:maxloc+1] if x1 < trough_mid]
print trough, len(trough), trough_mid

rim_mid = (z_smooth[xloc] + z_smooth[-1]) / 2.0
rim_mid_loc = np.where(z_smooth[10:] <= rim_mid)[-1]
print 'Rim mid loc',rim_mid_loc[-1]
rim = [x1 for x1 in z_smooth[:maxloc +1] if (x1 >=trough_mid) ]
for x1 in z_smooth[maxloc+1:]:
    if x1 > rim_mid:
        rim.append(x1)
print rim, len(rim), rim_mid

center = [round(x1,4) for x1 in z_smooth[maxloc:] if x1 <= rim_mid]
print center, len(center)

l1 = len(trough)
l2 = len(center)-2

peat = np.concatenate((trough, rim, center))
print len(x), len(peat)



In [ ]:
#Changing organic layer thickness 
#AreaC_OM = [center : 10 cm , rim : 10cm, trough : 16 cm]
def org_layer_bottom_bndry(topo_pos, i):
    if (topo_pos in trough and i <maxloc):
        thickness = 0.14
    elif(topo_pos in rim and i < rim_mid_loc[-1]):
        thickness = 0.08
    elif (topo_pos in center):
        thickness = 0.20
    return thickness

In [ ]:
peat_thickness = [org_layer_bottom_bndry(p,i) for i,p in enumerate(peat)]
fig, ax = plt.subplots()
ax.plot(x, peat, 'g*')
ax.plot(x,z_smooth, 'r.')
ax1 = ax.twinx()
ax1.plot(x,peat_thickness, 'c-')


In [11]:
outfile = "/Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/"
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []

for i in range(1):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1001)
    z = z + 0.02
    Z.append(z)
print ('Moss ', z)

for i in range(4):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1002)
    z = z + 0.02
    Z.append(z)
print ('Peat ', z)

for i in range(15): 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + 0.02
    Z.append(z)
print ('Upper mineral ', z)

dz = .02
for i in range(35):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print ('Ice rich', z)

for i in range(25):
    dz *= 1.12
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print (z)


layer_types.append('snapped')
layer_data.append(-45.0) # bottom location
layer_ncells.append(1)
layer_mat_ids.append(1003)

m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)

#m3.write_exodus(outfile + "areaC-updist-vladimir-om10cm.exo")
m3.write_exodus(outfile + "areaC-3layers-diver-om10cm-l.exo")
#m3.write_exodus(outfile + "column-3layers-om10cm.exo")

('Moss ', 0.02)
('Peat ', 0.1)
('Upper mineral ', 0.4000000000000001)
('Ice rich', 3.716409519999923)
41.2555869471

You are using exodus.py v 1.05 (beta-cmake), a python wrapper of some of the exodus II library.

Copyright (c) 2013, 2014, 2015, 2016, 2017 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.  

Opening exodus file: /Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/areaC-3layers-diver-om10cm-l.exo
Closing exodus file: /Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/areaC-3layers-diver-om10cm-l.exo


In [ ]:
#variable peat thickness

outfile = "/Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/"
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []

for i in range(1):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1001*np.ones((m2.num_cells(),),'i'))
    z = z + 0.02
    Z.append(z)
print ('Moss ', z)

for i in range(9):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(-1000*np.ones((m2.num_cells(),),'i'))
    z = z + 0.02
    Z.append(z)
print ('Peat ', z)

for i in range(10): 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))
    z = z + 0.02
    Z.append(z)
print ('Upper mineral ', z)

dz = .02
for i in range(35):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))
    z = z + dz
    Z.append(z)
print ('Ice rich', z)

for i in range(25):
    dz *= 1.12
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))
    z = z + dz
    Z.append(z)
print (z)


layer_types.append('snapped')
layer_data.append(-45.0) # bottom location
layer_ncells.append(1)
layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))


mat_ids=np.zeros((m2.num_cells(), 11), 'i')

for i in range(m2.num_cells()):
    for j in range(11):
        if (Z[j]<=peat_thickness[i]):
            mat_ids[i,j]=1002
        else:
            mat_ids[i,j]=1003
for j in range(11):
    layer_mat_ids[j] = mat_ids[:,j]

print len(layer_mat_ids), len(layer_ncells), len(layer_types), len(layer_data)


m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)


m3.write_exodus(outfile + "areaC-3layers-diver-om10cm.exo")






In [ ]:
# Old z resolution -- November 8, 2018
outfile = "/Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/"
# layer extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []

for i in range(2):
    layer_types.append('constant')
    layer_data.append(0.01)
    layer_ncells.append(1)
    layer_mat_ids.append(1001)
    z = z + 0.01
    Z.append(z)
print ('Moss ', z)

for i in range(4): 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1002)
    z = z + 0.02
    Z.append(z)
print ('Peat ', z)

for i in range(20): #8cm peat, n=20, 20cm peat n = 14
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + 0.02
    Z.append(z)
print (z)


dz = .02
for i in range(21):
    dz *= 1.05
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print (z)

for i in range(17):
    dz *= 1.2
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print (z)

for i in range(8):
    dz *= 1.2
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    #print ('5th layer',z)
    z = z + dz
    Z.append(z)
print (Z)
print len(Z)
layer_types.append('snapped')
layer_data.append(-40.0) # bottom location
layer_ncells.append(2)
layer_mat_ids.append(1003)

#print layer_data
m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)

#m3.write_exodus(outfile + "areaC-updist-vladimir-om10cm.exo")
m3.write_exodus(outfile + "areaC-smooth-om10cm.exo")